In [1]:
import trimesh

scene = trimesh.Scene()

In [4]:
mesh_remeshed = trimesh.load_mesh('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/template.off')
mesh_original = trimesh.load_mesh('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/template.ply')

# mesh_template.apply_transform(mesh_template.principal_inertia_transform)
mesh_remeshed.apply_transform(mesh_remeshed.principal_inertia_transform)
mesh_original.apply_transform(mesh_original.principal_inertia_transform)

In [ ]:
import numpy as np

scene.geometry.clear()


scene.add_geometry(mesh_remeshed)
scene.add_geometry(
    trimesh.Trimesh(vertices=mesh_original.vertices + np.array([1, 0, 0]), faces=mesh_original.faces)
)
scene.show()

In [14]:
import utils.fmap_util as fmap_util
import torch

mesh_reflected = mesh_original.copy()
mesh_reflected.vertices[:, 1] *= -1

# correspondence by a nearest neighbor search
corr = fmap_util.nn_query(
    torch.tensor(mesh_remeshed.vertices),
    torch.tensor(mesh_original.vertices), 
    )

corr_reflected = fmap_util.nn_query(
    torch.tensor(mesh_remeshed.vertices),
    torch.tensor(mesh_reflected.vertices), 
    )

In [16]:
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    
    torch.tensor(mesh_remeshed.vertices), torch.tensor(mesh_remeshed.faces),
    torch.tensor(mesh_original.vertices), torch.tensor(mesh_original.faces),
    corr_reflected,
    axes_color_gradient=[1],
    base_cmap='hsv'
)

scene.show()

In [17]:
# save the reflected correspondence to /home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed

np.savetxt('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/corr_symmetric.txt', corr_reflected.numpy(), fmt='%d')